# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.float_format', '{:.3f}'.format)  # Three digits past decimal
np.set_printoptions(precision=3, suppress=True)  # Suppress scientific notation

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (12, 8)

spam = pd.read_csv('https://jdgrossman.com/assets/spam.csv')

spam.sample(10)

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
3501,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.570,0.000,0.000,0.000,0.000,2.312,11,37,0
1017,0.150,0.150,0.300,0.000,0.750,0.000,0.000,0.000,0.000,0.000,...,0.058,0.145,0.000,0.087,0.000,0.000,1.540,18,208,1
3389,1.260,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.178,0.000,0.000,1.272,3,14,0
4492,0.000,0.000,1.090,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.191,0.000,0.000,0.000,0.000,3.000,15,51,0
1889,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,2.000,3,4,0
622,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.470,...,0.000,0.000,0.000,0.000,0.000,0.000,2.769,15,36,1
3836,0.000,0.000,0.820,0.000,0.820,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.119,0.000,0.000,1.272,6,42,0
1548,0.170,0.000,0.080,0.000,0.420,0.080,0.080,0.420,0.080,0.080,...,0.000,0.084,0.028,0.098,0.014,0.000,4.049,48,575,1
2335,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.500,3,6,0
2425,0.750,0.000,0.370,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.305,0.000,0.061,0.000,0.000,1.903,13,118,0


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [3]:

#1. How many emails are in the database?
print(f"Number of emails: {len(spam)}")

#2. What fraction of the emails in the database are spam?
spam_fraction = spam['is_spam'].mean()
print(f"Fraction of emails that are spam: {spam_fraction}")

# Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?
max_money_idx = spam['money'].idxmax()
max_money_pct = spam.loc[max_money_idx, 'money']
print(f"Email with highest 'money' percentage: index {max_money_idx}, percentage: {max_money_pct}")

Number of emails: 4601
Fraction of emails that are spam: 0.39404477287546186
Email with highest 'money' percentage: index 545, percentage: 12.5


####Number of emails: 4601
####Fraction of emails that are spam: 0.39404477287546186
####Email with highest 'money' percentage: index 545
####percentage: 12.5

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [4]:
import statsmodels.api as sm

# Select predictors
X = spam[['char_dollar', 'credit', 'money', 're']]
X = sm.add_constant(X)

# Response variable
y = spam['is_spam']

# Fit linear probability model
lpm_model = sm.OLS(y, X).fit()

print(lpm_model.summary())


                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        16:34:11   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Interpretation of the Intercept (β₀):The intercept represents the estimated probability that an email is spam when it contains 0% dollar signs, 0% “credit” words, 0% “money” words, and 0% “re” words.

---


Interpretation of the char_dollar Coefficient (β₁): For each 1-percentage-point increase in the percentage of characters that are $, the probability that an email is spam increases by β₁.

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [6]:
#Predict in-sample probabilities
spam['pred_prob'] = lpm_model.predict(X)

#Find the smallest and largest predicted probabilities
min_pred = spam['pred_prob'].min()
max_pred = spam['pred_prob'].max()

print("Smallest predicted probability:", min_pred)
print("Largest predicted probability:", max_pred)


Smallest predicted probability: -0.8125403869560923
Largest predicted probability: 3.8494089964108236


The smallest predicted probability is –0.81 and the largest is 3.85, which are impossible values for real probabilities. This shows that linear probability models can produce invalid predictions outside the [0, 1] range.

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [7]:
def prob_to_odds(p):
    return p / (1 - p)

def odds_to_prob(o):
    return o / (1 + o)

odds_test = prob_to_odds(2/3)
print(f"Odds for probability 2/3: {odds_test:.3f}")

prob_test = odds_to_prob(2)
print(f"Probability for 2:1 odds: {prob_test:.3f}")

# Suppose probability of winning is 60%. If I double my odds, what is new probability?
current_prob = 0.6
current_odds = prob_to_odds(current_prob)
new_odds = current_odds * 2
new_prob = odds_to_prob(new_odds)
print(f"New probability after doubling odds: {new_prob:.3f}")


Odds for probability 2/3: 2.000
Probability for 2:1 odds: 0.667
New probability after doubling odds: 0.750


####Odds for probability 2/3: 2.000
####Probability for 2:1 odds: 0.667
####New probability after doubling odds: 0.750

## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [9]:
logit_model = sm.Logit(y, sm.add_constant(X))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.481178
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                          Logit   Df Residuals:                     4596
Method:                           MLE   Df Model:                            4
Date:                Tue, 18 Nov 2025   Pseudo R-squ.:                  0.2824
Time:                        16:52:05   Log-Likelihood:                -2213.9
converged:                       True   LL-Null:                       -3085.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.0666      0.043    -24.680      0.000      -1.151      -0.982
char_dollar    11.8176    

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [10]:
coef = result.params

# 1. Log-odds interpretation
# Intercept: log-odds of spam when predictors are zero
# money: change in log-odds for a 1-unit increase in money
print("Logistic Regression Coefficients:")
for name, c in coef.items():
    print(f"{name}: {c:.3f}")

# 2. Odds interpretation
# Intercept: baseline odds
# money: odds multiplier per 1-unit increase
print("\nExponentiated coefficients (odds scale):")
for name, c in coef.items():
    print(f"{name}: {np.exp(c):.3f}")


# 3. Probability interpretation (approximate)
print("\nApproximate change in probability (divide by 4):")
for name, c in coef.items():
    print(f"{name}: {c/4:.3f}")




Logistic Regression Coefficients:
const: -1.067
char_dollar: 11.818
credit: 2.312
money: 1.993
re: -0.776

Exponentiated coefficients (odds scale):
const: 0.344
char_dollar: 135613.881
credit: 10.094
money: 7.340
re: 0.460

Approximate change in probability (divide by 4):
const: -0.267
char_dollar: 2.954
credit: 0.578
money: 0.498
re: -0.194
